![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that international students have a higher risk of mental health difficulties than the general population, and that social connectedness (belonging to a social group) and acculturative stress (stress associated with joining a new culture) are predictive of depression.


Explore the `students` data using PostgreSQL to find out if you would come to a similar conclusion for international students and see if the length of stay is a contributing factor.

Here is a data description of the columns you may find helpful.

| Field Name    | Description                                      |
| ------------- | ------------------------------------------------ |
| `inter_dom`     | Types of students (international or domestic)   |
| `japanese_cate` | Japanese language proficiency                    |
| `english_cate`  | English language proficiency                     |
| `academic`      | Current academic level (undergraduate or graduate) |
| `age`           | Current age of student                           |
| `stay`          | Current length of stay in years                  |
| `todep`         | Total score of depression (PHQ-9 test)           |
| `tosc`          | Total score of social connectedness (SCS test)   |
| `toas`          | Total score of acculturative stress (ASISS test) |

In [72]:
-- Run this code to view the data in students
SELECT * 
FROM students limit 10;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
5,Inter,SEA,Male,Grad,24,4,6,Long,3,Average,4,High,Yes,No,No,No,No,6,Mild,38,18,8,10,8,7,3,29,83,6,5,4,2,1,1,2,1,5,1,NaN,Yes,Yes,No,No,No,No,No,No,Yes,No,No
6,Inter,SA,Male,Grad,23,4,1,Short,3,Average,5,High,Yes,No,No,No,No,3,Min,46,17,6,10,5,3,2,15,58,7,5,7,2,2,1,5,1,1,1,NaN,Yes,Yes,Yes,No,No,No,Yes,No,No,No,No
7,Inter,SEA,Female,Grad,30,5,2,Medium,1,Low,1,Low,Yes,Yes,Yes,No,No,9,Mild,41,16,20,19,15,11,6,40,127,7,2,2,2,6,2,1,1,3,1,NaN,Yes,No,No,No,Yes,No,No,No,No,No,No
8,Inter,SEA,Female,Grad,25,4,4,Long,4,High,4,High,No,No,No,Yes,Other,7,Mild,36,22,12,13,13,10,6,33,109,4,4,4,4,4,4,4,4,4,4,NaN,No,No,No,No,No,No,No,No,No,No,No
9,Inter,Others,Male,Grad,31,5,2,Medium,1,Low,4,High,Yes,Yes,No,No,No,3,Min,48,8,4,5,12,3,2,17,51,1,1,1,2,1,1,2,1,2,1,NaN,No,No,No,No,No,No,No,No,No,No,No


# 🧠 Analyzing Students’ Mental Health using SQL

This project explores the relationship between **language proficiency**, **academic level**, and **length of stay** on students’ **mental health indicators** — including depression, stress, and social connectedness.

The analysis is based on a dataset from an international Japanese university survey (2018), available in the `students` table.

---

## 🎯 Objectives
1. Explore distributions of key categorical variables.
2. Compare average depression, stress, and social connectedness across groups.
3. Identify possible relationships between:
   - English proficiency ↔ Depression / Connectedness  
   - Academic level ↔ Stress  
   - Stay duration ↔ Depression / Stress
---
## 🧩 Tools Used
- **SQL** in DataCamp Datalab  
- **PostgreSQL** syntax  
- **Built-in visualization (Charts tab)** for bar and line plots

### Dataset Preview
Here we can see the first 10 records. Each row represents a student with attributes such as region, gender, academic level, language proficiency, and psychological test scores.

We'll focus mainly on:
- `stay_cate`
- `english_cate`
- `academic`
- `todep`, `toas`, `tosc`

In [73]:
-- Start coding here
SELECT * FROM STUDENTS
LIMIT 10;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No
5,Inter,SEA,Male,Grad,24,4,6,Long,3,Average,4,High,Yes,No,No,No,No,6,Mild,38,18,8,10,8,7,3,29,83,6,5,4,2,1,1,2,1,5,1,NaN,Yes,Yes,No,No,No,No,No,No,Yes,No,No
6,Inter,SA,Male,Grad,23,4,1,Short,3,Average,5,High,Yes,No,No,No,No,3,Min,46,17,6,10,5,3,2,15,58,7,5,7,2,2,1,5,1,1,1,NaN,Yes,Yes,Yes,No,No,No,Yes,No,No,No,No
7,Inter,SEA,Female,Grad,30,5,2,Medium,1,Low,1,Low,Yes,Yes,Yes,No,No,9,Mild,41,16,20,19,15,11,6,40,127,7,2,2,2,6,2,1,1,3,1,NaN,Yes,No,No,No,Yes,No,No,No,No,No,No
8,Inter,SEA,Female,Grad,25,4,4,Long,4,High,4,High,No,No,No,Yes,Other,7,Mild,36,22,12,13,13,10,6,33,109,4,4,4,4,4,4,4,4,4,4,NaN,No,No,No,No,No,No,No,No,No,No,No
9,Inter,Others,Male,Grad,31,5,2,Medium,1,Low,4,High,Yes,Yes,No,No,No,3,Min,48,8,4,5,12,3,2,17,51,1,1,1,2,1,1,2,1,2,1,NaN,No,No,No,No,No,No,No,No,No,No,No


### 📈 Interpreting Distributions
These simple aggregations help understand how balanced the dataset is.  
For example:
- If most students are in `Short` stay, the results may be skewed.
- Balanced categories = more reliable comparisons.

In [74]:
-- 1. Distribution of stay duration
SELECT CASE 
			WHEN stay_cate is null or trim(stay_cate) = ''
				then 'Unknown' 
			ELSE stay_cate
			end as stay_cate,
	COUNT(*) AS total_students
FROM students
GROUP BY CASE 
			WHEN stay_cate is null or trim(stay_cate) = ''
				then 'Unknown'
			ELSE stay_cate
			end
ORDER BY stay_cate;

,stay_cate,total_students
0,Long,32
1,Medium,121
2,Short,115
3,Unknown,18


,stay_cate,total_students
0,Long,32
1,Medium,121
2,Short,115
3,Unknown,18


In [76]:
-- 2. Distribution of English proficiency
SELECT 
    CASE 
        WHEN english_cate IS NULL OR TRIM(english_cate) = '' THEN 'Unknown'
        ELSE english_cate
    END AS english_cate,
    COUNT(*) AS total_students
FROM students
GROUP BY CASE 
        WHEN english_cate IS NULL OR TRIM(english_cate) = '' THEN 'Unknown'
        ELSE english_cate
    END
ORDER BY english_cate;

,english_cate,total_students
0,Average,80
1,High,166
2,Low,22
3,Unknown,18


,english_cate,total_students
0,Average,80
1,High,166
2,Low,22
3,Unknown,18


In [78]:
-- 3. Distribution of academic level
SELECT 
	CASE 
		WHEN academic IS NULL OR TRIM(academic) = '' then 'Unknown'
		ELSE academic
	END as academic ,
	COUNT(*) AS Total_students
FROM students
GROUP BY CASE 
			WHEN academic IS NULL OR TRIM(academic)='' THEN 'Unknown'
			ELSE academic 
		END 
ORDER BY academic;

,academic,total_students
0,Grad,21
1,Under,247
2,Unknown,18


,academic,total_students
0,Grad,21
1,Under,247
2,Unknown,18


## **Analyzing the relationships between psychological factors and socio-academic variables.**

## Relationship 1 – Stay Duration vs Mental Health 
**Longer stays are associated with slightly higher stress and lower social connectedness,
while depression levels remain relatively stable across groups**.

In [80]:
-- Relationship 1: Stay Duration vs Mental Health
SELECT  
    stay_category,
    ROUND(AVG(todep), 2) AS avg_depression,
    ROUND(AVG(toas), 2) AS avg_stress,
    ROUND(AVG(tosc), 2) AS avg_connectedness
FROM (
    SELECT  
        CASE 
            WHEN stay_cate IS NULL OR TRIM(stay_cate) = '' THEN 'Unknown'
            ELSE stay_cate
        END AS stay_category,
        todep,
        toas,
        tosc
    FROM students
) AS cleaned
GROUP BY stay_category
ORDER BY 
    CASE stay_category
        WHEN 'Short' THEN 1
        WHEN 'Medium' THEN 2
        WHEN 'Long' THEN 3
        WHEN 'Unknown' THEN 4
        ELSE 5
    END;

,stay_category,avg_depression,avg_stress,avg_connectedness
0,Short,7.70,71.03,37.94
1,Medium,8.74,72.86,37.48
2,Long,7.84,75.44,35.78
3,Unknown,NaN,NaN,NaN


,stay_category,avg_depression,avg_stress,avg_connectedness
0,Short,7.70,71.03,37.94
1,Medium,8.74,72.86,37.48
2,Long,7.84,75.44,35.78
3,Unknown,NaN,NaN,NaN


### Summary
Students with **longer stays** show the **highest connectedness (75.4)** and **lowest depression (7.7)**.

### **Relationship 2: English Proficiency vs Mental Health**

In [82]:
-- Relationship 2: English Proficiency vs Mental Health
SELECT
    english_category,
    ROUND(AVG(todep), 2) AS avg_depression,
    ROUND(AVG(toas), 2) AS avg_stress,
    ROUND(AVG(tosc), 2) AS avg_connectedness
FROM (
    SELECT 
        CASE 
            WHEN english_cate IS NULL OR TRIM(english_cate) = '' THEN 'Unknown'
            ELSE english_cate
        END AS english_category,
        todep,
        toas,
        tosc
    FROM students
) AS cleaned
GROUP BY english_category
ORDER BY 
    CASE english_category
        WHEN 'Low' THEN 1
        WHEN 'Average' THEN 2
        WHEN 'High' THEN 3
        WHEN 'Unknown' THEN 4
        ELSE 5
    END;

,english_category,avg_depression,avg_stress,avg_connectedness
0,Low,9.32,68.36,37.05
1,Average,8.31,67.44,38.48
2,High,7.98,75.30,37.05
3,Unknown,NaN,NaN,NaN


,english_category,avg_depression,avg_stress,avg_connectedness
0,Low,9.32,68.36,37.05
1,Average,8.31,67.44,38.48
2,High,7.98,75.30,37.05
3,Unknown,NaN,NaN,NaN


**Summery**

Students with higher English proficiency show lower depression levels but higher stress, possibly due to academic pressure.
Social connectedness remains relatively stable across proficiency levels.

**Relationship 3: Academic Level vs Mental Health**

In [84]:
-- Relationship 3: Academic Level vs Mental Health
SELECT  
    academic_level,
    ROUND(AVG(todep), 2) AS avg_depression,
    ROUND(AVG(toas), 2) AS avg_stress,
    ROUND(AVG(tosc), 2) AS avg_connectedness
FROM (
    SELECT  
        CASE 
            WHEN academic IS NULL OR TRIM(academic) = '' THEN 'Unknown'
            ELSE academic
        END AS academic_level,
        todep,
        toas,
        tosc
    FROM students
) AS cleaned
GROUP BY academic_level
ORDER BY 
    CASE academic_level
        WHEN 'Under' THEN 1
        WHEN 'Grad' THEN 2
        WHEN 'Unknown' THEN 3
        ELSE 4
    END;

,academic_level,avg_depression,avg_stress,avg_connectedness
0,Under,8.43,72.07,37.16
1,Grad,5.29,76.05,41.19
2,Unknown,NaN,NaN,NaN


,academic_level,avg_depression,avg_stress,avg_connectedness
0,Under,8.43,72.07,37.16
1,Grad,5.29,76.05,41.19
2,Unknown,NaN,NaN,NaN


**Summery**  
Graduate students show **lower depression** (5.29) but **higher stress** (76.05) and **stronger social connectedness** (41.19) compared to undergraduates.  
This suggests that while graduate students may handle emotional challenges better, they experience higher pressure and maintain more academic or social engagement.

## **FINAL: Stay (length) impact on mental health for INTERNATIONAL students**

In [86]:
-- FINAL: Stay (length) impact on mental health for INTERNATIONAL students
SELECT
  stay AS stay,
  COUNT(*) AS count_int,
  ROUND(AVG(todep), 2) AS average_phq,
  ROUND(AVG(tosc), 2) AS average_scs,
  ROUND(AVG(toas), 2) AS average_as
FROM students
WHERE inter_dom = 'Inter'
  AND stay IS NOT NULL
  AND TRIM(stay::text) <> ''
GROUP BY stay
ORDER BY stay DESC;

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


In [87]:
df = _
df

,stay,count_int,average_phq,average_scs,average_as
0,10,1,13.00,32.00,50.00
1,8,1,10.00,44.00,65.00
2,7,1,4.00,48.00,45.00
3,6,3,6.00,38.00,58.67
4,5,1,0.00,34.00,91.00
5,4,14,8.57,33.93,87.71
6,3,46,9.09,37.13,78.00
7,2,39,8.28,37.08,77.67
8,1,95,7.48,38.11,72.80


In [88]:
df.shape

(9, 5)